In [2]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from prec_recall import prec_recall
# fix random seed for reproducibility
np.random.seed(7)

Using TensorFlow backend.
/Users/wenqi/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [4]:
# load pima indians dataset

if dataset == "subsample":
    X_train = np.load("../data/subsamp_data/processed_X_train.npy")
    y_train = np.load("../data/subsamp_data/processed_y_train.npy")
elif dataset == "origin":
    X_train = np.load("../data/origin_data/X_train.npy")
    y_train = np.load("../data/origin_data/y_train.npy")
else:
    raise Exception("Unknown dataset name")

# val / test always on the largest dataset
X_val = np.load("../data/origin_data/X_val.npy")
y_val = np.load("../data/origin_data/y_val.npy")
X_test = np.load("../data/origin_data/X_test.npy")
y_test = np.load("../data/origin_data/y_test.npy")

print("shape: {}".format(X_val.shape))
print(X_val)
print(X_val[0])

shape: (56962, 28)
[[-1.88113802 -0.74235614  1.06995965 ...  0.92028053  0.08424699
   0.02944781]
 [ 1.22525413  0.34181835  0.75873135 ...  0.05821159 -0.02147733
   0.02647026]
 [ 1.18797456 -0.87555093  0.53915157 ... -0.14552036  0.09054168
   0.03467882]
 ...
 [-1.14996324  1.69646167  1.63711373 ...  0.21034302 -0.16204654
   0.03119279]
 [ 1.75455368 -0.69939829 -0.07633214 ...  0.10463621 -0.00585291
  -0.01962208]
 [-0.70763501  0.49330227  2.64808871 ... -0.39874131 -0.38558861
  -0.20558893]]
[-1.88113802 -0.74235614  1.06995965 -1.23734802 -1.47560302  0.19913695
  1.86328489 -0.47423259  0.4101315  -1.24941593 -0.5773685   0.14005026
  0.85590515 -0.40157396  1.35768666  0.14682466 -0.4378473   0.14872055
  0.57545165 -0.33199204  0.02818773  0.67043208 -0.225319    0.06510109
  0.47225463  0.92028053  0.08424699  0.02944781]


In [3]:
# create model
model = Sequential()
model.add(Dense(12, input_dim=28, activation='relu'))
model.add(Dense(6, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
# Fit the model
class_weight = {0: 1,
                1: 5}
model.fit(X_train, y_train, epochs=10, batch_size=10, class_weight=class_weight)

Epoch 1/10
622/622 [==============================] - 0s 131us/step - loss: 0.8100 - acc: 0.5048
Epoch 2/10
622/622 [==============================] - 0s 116us/step - loss: 0.7742 - acc: 0.5080
Epoch 3/10
622/622 [==============================] - 0s 107us/step - loss: 0.7390 - acc: 0.5096
Epoch 4/10
622/622 [==============================] - 0s 120us/step - loss: 0.7070 - acc: 0.5161
Epoch 5/10
622/622 [==============================] - 0s 104us/step - loss: 0.6753 - acc: 0.5273
Epoch 6/10
622/622 [==============================] - 0s 91us/step - loss: 0.6488 - acc: 0.5354
Epoch 7/10
622/622 [==============================] - 0s 92us/step - loss: 0.6245 - acc: 0.5611
Epoch 8/10
622/622 [==============================] - 0s 92us/step - loss: 0.6018 - acc: 0.5836
Epoch 9/10
622/622 [==============================] - 0s 92us/step - loss: 0.5798 - acc: 0.6077
Epoch 10/10
622/622 [==============================] - 0s 93us/step - loss: 0.5583 - acc: 0.6367


In [14]:
model.save_weights("models/model.h5")

In [10]:
def binary_label(labels):
    ans = []
    for i in labels:
        if i > 0.5:
            ans.append(1)
        else:
            ans.append(0)
            
    return ans

In [11]:
from prec_recall import prec_recall

print("validation accuracy:")
scores = model.evaluate(X_val, y_val)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
y_val_pred =  binary_label(model.predict(X_val))
prec_recall(label=y_val, predict=y_val_pred)

scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
y_test_pred =  binary_label(model.predict(X_test))
prec_recall(label=y_test, predict=y_test_pred)

validation accuracy:
188/188 [==============================] - 0s 25us/step

acc: 62.23%
TP count:    92
FP count:    69
TN count:    25
FN count:    2
Precision rate:  0.5714285714285714
Recall rate: 0.9787234042553191
174/174 [==============================] - 0s 21us/step

acc: 63.79%
TP count:    87
FP count:    63
TN count:    24
FN count:    0
Precision rate:  0.58
Recall rate: 1.0


In [12]:
print(y_val_pred)

[1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
# calculate predictions
predictions = model.predict(X)
# round predictions
rounded = [round(x[0]) for x in predictions]
print(rounded)

In [ ]:
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.load_weights("model.h5")

In [ ]:
scores = model.evaluate(X, Y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))